# Libraries

In [1]:
cd /home/tvangraft/tudelft/thesis/metaengineering

/home/tvangraft/tudelft/thesis/metaengineering


In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from src.pipeline.dataloader import DataLoader
from src.orchestrator.orchestrator import SklearnOrchestrator

from src.settings.strategy import Strategy
from src.settings.tier import Tier

from src.utils.utils import build_config

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [4]:
def get_dl_config_for_strategy(tier: Tier):
    lookup = {
        Tier.TIER0: dict(
            additional_filters=["is_precursor",],
            additional_transforms=["log_fold_change_protein",]
        ),
        Tier.TIER1: dict(
            additional_frames=["interaction_frame",],
            additional_filters=[
                "is_precursor",
            ],
            additional_transforms=[
                "log_fold_change_protein",
                "ppi_coo_matrix",
            ]
        )
    }
    return lookup.get(tier)

In [5]:
params = {
    'SVR': {
        'regressor__regressor': SVR(),
        'regressor__regressor__kernel': ['rbf', 'sigmoid'],
        'regressor__regressor__gamma': ['auto', 'scale'],
        'regressor__regressor__epsilon': [0.1, 0.01, 0.001, 0.0001],
        'regressor__regressor__C': [10, 100, 1000],
        'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler()],
    },
    'RandomForestRegressor': {
        'regressor__regressor': RandomForestRegressor(),
        'regressor__regressor__n_estimators': [10, 25, 50, 75, 100],
        'regressor__regressor__criterion': ['squared_error', 'friedman_mse'],
        'regressor__regressor__max_depth': [5, 10, 20],
        'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler()]
    },
    'ElasticNet': {
        'regressor__regressor': ElasticNet(),
        'regressor__regressor__l1_ratio': [0.01, 0.25, 0.5, 0.75, 1],
        'regressor__regressor__tol': [0.01],
        'regressor__preprocessor__num__scaler': [MinMaxScaler(), StandardScaler()]
    },
    'MLPRegressor': {
        'regressor__regressor': MLPRegressor(),
        'regressor__regressor__hidden_layer_sizes':  [[128, 32, 32], [64, 32]],
        'regressor__regressor__batch_size': [8, 16, 4, 2],
    }
}

strategies = [Strategy.ALL, Strategy.ONE_VS_ALL, Strategy.METABOLITE_CENTRIC]
tiers = [Tier.TIER0, Tier.TIER1]

In [7]:
DataLoader.DATA_FOLDER = './data/training/'
for tier in tiers:
    for strategy in strategies:
        print(f"{tier} - {strategy}")
        orchestrator: SklearnOrchestrator = SklearnOrchestrator()
        orchestrator.prepare_orchestrator(
            *build_config(
                strategy=strategy,
                tier=tier,
                params=params,
                forced_training=False,
                forced_testing=True,
                **get_dl_config_for_strategy(tier)
            )
        )
        orchestrator.run()

Tier.TIER0 - Strategy.ALL
ParsedDataLoaderConfig(additional_frames=[], additional_transforms=[<bound method FrameTransformers.log_fold_change_protein of <src.pipeline.frame.transformer.FrameTransformers object at 0x7fc58b164610>>], additional_filters=[<bound method FrameFilters.is_precursor of <src.pipeline.frame.filter.FrameFilters object at 0x7fc58c9b2a30>>])
Result for Strategy.ALL_all already exists
['all', 'SVR()', 'RandomForestRegressor()', 'ElasticNet()', 'MLPRegressor()']
architecture='all'
ParsedDataLoaderConfig(additional_frames=[], additional_transforms=[<bound method FrameTransformers.log_fold_change_protein of <src.pipeline.frame.transformer.FrameTransformers object at 0x7fc58b164610>>], additional_filters=[<bound method FrameFilters.is_precursor of <src.pipeline.frame.filter.FrameFilters object at 0x7fc58c9b2a30>>])
RandomForest model
RandomForest model
RandomForest model
RandomForest model
RandomForest model
RandomForest model
RandomForest model
RandomForest model
Random

KeyError: 'y_true'